# Classification of MNIST dataset digits
## Labels = 10
## Training Samples =  60,000
## Testing Samples =   10,000

In [1]:
# Modules
from PIL import Image
import dhash
import numpy as np
import scipy

from keras.datasets import mnist
from keras import utils

import matplotlib.pyplot as plt; %matplotlib inline

from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV, tree, metrics
from sklearn.ensemble import RandomForestClassifier


np.random.seed(123)  # for reproducibility


SyntaxError: invalid syntax (<ipython-input-1-2dbc32e12821>, line 10)

## 2. Load MNIST data from keras

In [ ]:
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

In [ ]:
plt.figure(figsize=(5,5))
for i in range(9):
    plt.subplot(3,3,1+i)
    plt.imshow(X_train[i], cmap = 'plasma')
    plt.title(  str(Y_train[i])  )

## 3. Image Hashing on rows and columns  for training set

In [ ]:
# size of image and output format
size = 9
format_ = '0' + str(size**2) + 'b'
X = X_train

#preallocate
X_train_hashed = np.zeros((X.shape[0], size**2 * 2));

for idx , Img in enumerate(X):
    row, col = dhash.dhash_row_col( Image.fromarray(Img) , size = size)
    hash_ = format(row, format_) + format(col, format_)
    # hash_ is string
    for colidx,num in enumerate(hash_):
        X_train_hashed[idx,colidx] = int(num)

## 4. Image Hashing on rows and columns  for training set

In [ ]:
X = X_test

X_test_hashed = np.zeros((X.shape[0], size**2 * 2));

for idx , Img in enumerate(X):
    row, col = dhash.dhash_row_col( Image.fromarray(Img) , size = size)
    hash_ = format(row, format_) + format(col, format_)
    
    # hash_ is string
    for col,num in enumerate(hash_):
        X_test_hashed[idx,col] = int(num)

## 5. Default outcome encoding and random forest classifier

In [ ]:
Yreal = Y_test
ytrain = Y_train

# train
RFC = RandomForestClassifier(n_estimators=100)
RFC = RFC.fit(X_train_hashed, ytrain)

# predict
y_pred = RFC.predict(X_test_hashed)

# evaluate
print("Classification Report:\n%s" % metrics.classification_report(Yreal, y_pred))

## 6. OneHot outcome encoding and random forest classifier

In [ ]:
Yreal = utils.to_categorical(Y_test)
ytrain = utils.to_categorical(Y_train)

# train
RFC = RandomForestClassifier(n_estimators=100)
RFC = RFC.fit(X_train_hashed, ytrain)

# predict
y_pred = RFC.predict(X_test_hashed)

# evaluate
print("Classification Report:\n%s" % metrics.classification_report(Yreal, y_pred))